# LangChain: RAG

Colab Notebook:
https://colab.research.google.com/drive/1rPTvZM4HO--UNKHQ3CfDV7p3_zhRIX_j?usp=sharing

In [25]:
!pip install  langchain langchain-community langchainhub langchain-openai chromadb bs4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/97/cd/1c618f89d3fcbb375c99a3ea950bffba8a01862cc0f0ab5032dfb95e8d1e/langsmith-0.0.92-py3-none-any.whl (56 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 1.4 MB/s eta 0:00:00a

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
# from langchain.prompts import ChatPromptTemplate



In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [5]:
llm = AzureChatOpenAI(temperature=0.0, openai_api_version="2023-12-01-preview", azure_deployment="gpt-35-turbo-felix", openai_api_key=os.getenv("AZURE_OPENAI_KEY"))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002-felix",
    openai_api_version="2023-12-01-preview",
)

In [7]:
template = """使用以下内容回答最后的问题。请使用中文回答
如果你不知道答案，就说你不知道，不要试图编造答案。
最多使用三句话，并尽可能简明扼要。
总是在回答的最后说“谢谢你的提问！”

{context}

问题是: {question}

有用的回答:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [8]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://export.shobserver.com/baijiahao/html/715336.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("titleFont", "contentFont")
        )
    ),
)
docs = loader.load()
docs


[Document(page_content='C919龙年首飞，乘务长带着自豪感迎接双向客流2月10日早上8时15分，东航MU9197航班从上海虹桥机场起飞，飞往成都天府机场。这是国产大飞机C919的龙年首飞。“今天是大年初一，是中国传统的新春佳节。 在这个喜庆祥和的节日里，我们全体机组成员向您致以最诚挚的问候……”飞机上，乘务长娄颖雯开始播报。今年是C919首次服务春运。目前已经投运的C919飞机共有四架，都属于东航，分别在京沪、沪蓉航线执飞。娄颖雯介绍，龙年首个航班，C919飞机客座率超过九成，大多是举家出行的旅客，有的带着小孩，有的带着老人。不少旅客第一次乘坐国产大飞机，激动的情绪难以掩饰。有老人感慨：“没想到有生之年可以乘坐中国自己造的大飞机。”小朋友们也十分兴奋，因为大年初一，等飞机平飞后，每个C919航班上都会举行“龙腾闹新春”活动。乘务组会给旅客发福袋、玩偶等，并为他们拍照留念。此外，龙年到了，C919飞机上的餐食也别具一格。龙造型的“小笼包”和甜品让旅客直呼用心。而红豆味的大白兔牛奶又充满上海特色，让旅客印象深刻。娄颖雯是第一批C919乘务员，2023年5月28日，东航首架C919进行商业首飞时，她就在飞机上。“以前飞的都是波音和空客，现在能飞自己国家的飞机，确实很幸运，也感到很自豪。”她还记得自己曾询问父亲的意见，父亲说：“去吧，我支持你。”这也让她充满了力量。自C919开始执飞商业航班以来，几乎每天都会有旅客问“这是首架飞机吗？”“国产大飞机有什么特色？”这时，娄颖雯就会情不自禁地介绍起来。“你可以看航班号，C919A是首架飞机，机身上有首架机的印章。C919的过道比波音、空客宽，并且客舱三个座位中间的那个座位，比两边座位宽1.5厘米，坐在中间也挺舒适。此外，飞机的氛围灯会根据不同场景切换，在迎送客时会选用彩虹灯；洗手间水温有8档调温；2.25米的过道高度也比一般飞机高……”当天，娄颖雯飞抵成都后，又跟随后续的C919航班飞回上海。在回程航班上，飞机客座率依然超过九成。“C919的客座率一直都很高，大家都希望打卡。此外，2023年以来，一个明显的趋势是，双向出行的旅游客流量很大，不像以前许多线路只有去程人多、返程人少。”娄颖雯说。新年的C919，见证了中国旅游经济的活力。栏目主编：李晔本文作者：王力题图来源：采访对象提供图片来源：采访对象提供'

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=64)
splits = text_splitter.split_documents(docs)

splits


[Document(page_content='C919龙年首飞，乘务长带着自豪感迎接双向客流2月10日早上8时15分，东航MU9197航班从上海虹桥机场起飞，飞往成都天府机场。这是国产大飞机C919的龙年首飞。“今天是大年初一，是中国传统的新春佳节。 在这个喜庆祥和的节日里，我们全体机组成员向您致以最诚挚的问候……”飞机上，乘务长娄颖雯开始播报。今年是C919首次服务春运。目前已经投运的C919飞机共有四架，都属于东航，分别在京沪、沪蓉航线执飞。娄颖雯介绍，龙年首个航班，C919飞机客座率超过九成，大多是举家出行的旅客，有的带着小孩，有的带着老人。不少旅客第一次乘坐国产大飞机，激动的情绪难以掩饰。有老人感慨：“没想到有生之年可以乘坐中国自己造的大飞机。”小朋友们也十分兴奋，因为大年初一，等飞机平飞后，每个C919航班上都会举行“龙腾闹新春”活动。乘务组会给旅客发福袋、玩偶等，并为他们拍照留念。此外，龙年到了，C919飞机上的餐食也别具一格。龙造型的“小笼包”和甜品让旅客直呼用心。而红豆味的大白兔牛奶又充满上海特色，让旅客印象深刻。娄颖雯是第一批C919乘务员，2023年5月28日，东航首架C919进行商业首飞时，她就在飞机上。“以前飞的都是波音和空客，现在能飞自己国家的飞机，确实很幸运，也感到很自豪。”她还记得自己曾询问父亲的意见，父亲说：“去吧，我支持你。”这也让她充满了力量。自C919开始执飞商业航班以来，几乎每天都会有旅客问“这是首架飞机吗？”“国产大飞机有什么特色？”这时，娄颖雯就会情不自禁地介绍起来。“你可以看航班号，C919A是首架飞机，机身上有首架机的印章。C919的过道比波音、空客宽，并且客舱三个座位中间的那个座位，比两边座位宽1.5厘米，坐在中间也挺舒适。此外，飞机的氛围灯会根据不同场景切换，在迎送客时会选用彩虹灯；洗手间水温有8档调温；2.25米的过道高度也比一般飞机高……”当天，娄颖雯飞抵成都后，又跟随后续的C919航班飞回上海。在回程航班上，飞机客座率依然超过九成。“C919的客座率一直都很高，大家都希望打卡。此外，2023年以来，一个明显的趋势是，双向出行的旅游客流量很大，不像以前许多线路只有去程人多、返程人少。”娄颖雯说。新年的C919，见证了中国旅游经济的活力。栏目主编：李晔本文作者：王力题图来源：采访对象提供图片来源：采访对象提供'

In [10]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [13]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_kwargus={"k": 2})

In [17]:

# prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("结合乘务长娄颖雯的介绍，回答C919大飞机的客舱过道有什么特点？")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'C919大飞机的客舱过道比波音、空客宽，并且客舱三个座位中间的那个座位比两边座位宽1.5厘米，坐在中间也挺舒适。谢谢你的提问！'

In [14]:
rag_chain.invoke("C919飞机上的餐食有什么特点？")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'C919飞机上的餐食有龙造型的“小笼包”和甜品，以及红豆味的大白兔牛奶，让旅客直呼用心，充满上海特色。谢谢你的提问！'

In [15]:
rag_chain.invoke("娄颖雯是什么时候开始在C919工作的")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'娄颖雯是第一批C919乘务员，具体时间不详。谢谢你的提问！'

In [19]:
rag_chain.invoke("已经投运的C919飞机共有几架？")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'已经投运的C919飞机共有四架。谢谢你的提问！'